# LH 임대주택 RAG 챗봇 테스트

PostgreSQL + pgvector 기반 하이브리드 검색 RAG 챗봇

In [1]:
import sys
sys.path.insert(0, '.')

from dotenv import load_dotenv
load_dotenv()

from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

from db_handler import hybrid_search
from embedder import embed_text

## 1. 검색 함수 정의

In [2]:
def retrieve_context(query: str, top_k: int = 10) -> str:
    """
    하이브리드 검색으로 관련 문서 검색
    (공고가 특정되면 해당 공고의 테이블 청크도 모두 포함)
    """
    # 쿼리 임베딩 생성
    query_embedding = embed_text(query)
    
    # 하이브리드 검색 (키워드 + 벡터 + RRF)
    results = hybrid_search(query, query_embedding, top_k=top_k)
    
    if not results:
        return "관련 정보를 찾을 수 없습니다."
    
    print(f"[검색 결과: {len(results)}개 청크]")  # 디버깅용
    
    # 컨텍스트 조합
    contexts = []
    for r in results:
        chunk_id, chunk_text, elem_type, table_ctx, title, region, category, similarity = r
        
        # 출처 정보
        source = f"[{title[:30]}...]"
        if table_ctx:
            source += f" ({table_ctx})"
        
        contexts.append(f"{source}\n{chunk_text}")
    
    return "\n\n---\n\n".join(contexts)

## 2. LLM 및 프롬프트 설정

In [3]:
# LLM 초기화
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0
)

# RAG 프롬프트
prompt = ChatPromptTemplate.from_template("""
당신은 LH(한국토지주택공사) 임대주택 공고 안내 전문가입니다.
아래 검색된 문서를 참고하여 사용자의 질문에 정확하게 답변해주세요.

## 검색된 문서
{context}

## 질문
{question}

## 답변 지침
- 검색된 문서에 있는 정보만 사용하세요.
- 숫자(금액, 비율, 날짜 등)는 정확하게 인용하세요.
- 여러 공고의 정보가 있으면 구분해서 설명하세요.
- 정보가 없으면 "해당 정보를 찾을 수 없습니다"라고 답변하세요.
- 답변 마지막에 "참조 공고:"를 추가하고, 답변에 사용한 공고명을 나열하세요.

답변:
""")

## 3. RAG 체인 구성

In [4]:
# RAG 체인
rag_chain = (
    {
        "context": lambda x: retrieve_context(x["question"]),
        "question": lambda x: x["question"]
    }
    | prompt
    | llm
    | StrOutputParser()
)

def ask(question: str) -> str:
    """질문하고 답변 받기"""
    return rag_chain.invoke({"question": question})

## 4. 테스트

In [5]:
question = "주거약자란 무엇인가요?"
print(f"Q: {question}\n")
print(f"A: {ask(question)}")

Q: 주거약자란 무엇인가요?

[검색 결과: 10개 청크]
A: 주거약자란 주거에 어려움을 겪고 있는 사람들을 의미하며, 주거약자용 주택에 신청할 수 있는 자격을 가진 사람들입니다. 주거약자에는 장애인 등이 포함되며, 이들은 주거약자용 주택에 설치된 편의시설을 이용할 수 있습니다. 주거약자용 주택은 해당 주거약자나 주거약자가 세대원으로 있는 경우에 신청할 수 있으며, 제공대상별 편의시설이 설치되어 있습니다.

참조 공고: [정정공고]파주운정3 A24BL 영구임대주택 추가 입주, [신규모집]부천원종 A2블록 행복주택 입주자모집, [시흥시 국민임대주택 예비입주자 모집(2025.11.05...), [정정공고]파주시 지역 국민임대주택 예비입주자 모집..., [의왕포일1블럭 창업지원주택 예비입주자 모집..., [정정공고]광명시 국민임대주택 예비입주자 모집(2025..., [연천전곡1단지 국민임대주택 예비입주자모집 공고..., [평택시.안성시 지역 국민임대 예비입주자 모집공고(202..., [의왕포일1블럭 창업지원주택 예비입주자 모집...


In [16]:
# 테스트 질문 1: 소득 기준
question = "서울에 나와있는 공고있어? 서울과 가까운 곳도 좋아"
print(f"Q: {question}\n")
print(f"A: {ask(question)}")

Q: 서울에 나와있는 공고있어? 서울과 가까운 곳도 좋아

[검색 결과: 23개 청크]
A: 현재 서울에 있는 공고는 "서울오류 행복주택 예비입주자 모집공고"가 있습니다. 이 공고는 2025년 5월 15일에 입주자 모집이 시작되며, 신청자격은 다음과 같습니다:

1. **대학생 계층**: 대학에 재학 중이거나 다음 학기에 입·복학 예정인 사람.
2. **청년 계층**: 19세 이상 39세 이하인 사람(1985.05.16.~2006.05.15).
3. **신혼부부·한부모가족 계층**: 공고일 현재 혼인 중인 사람으로서 혼인기간이 7년 이내이거나 6세 이하 자녀를 둔 사람.
4. **고령자**: 65세 이상의 사람(1960.05.15. 이전 출생).
5. **주거급여수급자**: 주거급여법에 따른 수급권자 또는 수급자.

신청은 LH청약플러스 홈페이지(https://apply.lh.or.kr) 또는 모바일 앱(LH청약플러스)에서 가능하며, 65세 이상의 고령자 등 인터넷 접수가 어려운 분들은 현장에서 접수할 수 있습니다.

또한, 서울과 가까운 지역으로는 "화성시 행복주택 입주자격완화 예비입주자 모집"이 있습니다. 화성시는 수도권 지역으로, 서울과의 접근성이 좋습니다.

참조 공고: 서울오류 행복주택 예비입주자 모집공고, 화성시 행복주택 입주자격완화 예비입주자 모집.


In [7]:
# 테스트 질문 2: 임대 조건
question = "서율 오류 공고의 주택 면적을 상세하게 알려주세요."
print(f"Q: {question}\n")
print(f"A: {ask(question)}")

Q: 서율 오류 공고의 주택 면적을 상세하게 알려주세요.

[검색 결과: 30개 청크]
A: 해당 공고의 주택 면적에 대한 정보는 다음과 같습니다:

1. **문산선유3**
   - 주택형: 46㎡
     - 세대당 계약면적(주거 전용): 46.63㎡
     - 세대당 계약면적(주거 공용): 19.9414㎡
     - 세대당 계약면적(기타 공용): 14.9920㎡
     - 세대당 계약면적(합계): 81.5634㎡
   - 주택형: 51㎡
     - 세대당 계약면적(주거 전용): 51.93㎡
     - 세대당 계약면적(주거 공용): 22.2080㎡
     - 세대당 계약면적(기타 공용): 16.6959㎡
     - 세대당 계약면적(합계): 90.8339㎡

2. **문산선유5**
   - 주택형: 46㎡
     - 세대당 계약면적(주거 전용): 46.61㎡
     - 세대당 계약면적(주거 공용): 21.9599㎡
     - 세대당 계약면적(기타 공용): 16.9569㎡
     - 세대당 계약면적(합계): 85.5268㎡
   - 주택형: 51㎡
     - 세대당 계약면적(주거 전용): 51.93㎡
     - 세대당 계약면적(주거 공용): 24.4663㎡
     - 세대당 계약면적(기타 공용): 18.8924㎡
     - 세대당 계약면적(합계): 95.2887㎡

3. **파주파주1**
   - 주택형: 39㎡
     - 세대당 계약면적(주거 전용): 39.57㎡
     - 세대당 계약면적(주거 공용): 18.8917㎡
     - 세대당 계약면적(기타 공용): 13.0538㎡
     - 세대당 계약면적(합계): 71.5155㎡
   - 주택형: 51㎡
     - 세대당 계약면적(주거 전용): 51.60㎡
     - 세대당 계약면적(주거 공용): 24.6351㎡
     - 세대당 계약면적(기타 공용): 17.0224㎡
     - 세대당 계약면적(합계): 93.2575㎡

4. **파주교하1**
   - 주택형: 46.73

In [8]:
# 테스트 질문 3: 신청 서류
question = "서울 오류 행복주택 공고의 신청자격에 대해서 알려주세요."
print(f"Q: {question}\n")
print(f"A: {ask(question)}")

Q: 서울 오류 행복주택 공고의 신청자격에 대해서 알려주세요.

[검색 결과: 27개 청크]
A: 서울 오류 행복주택의 신청자격은 다음과 같습니다:

1. **대학생 계층**
   - 대학에 재학 중이거나 다음 학기에 입·복학 예정인 사람
   - 취업준비생: 대학 또는 고등학교를 졸업(또는 중퇴)한 지 2년 이내인 사람

2. **청년 계층**
   - 청년: 19세 이상 39세 이하인 사람 (1985.05.16.~2006.05.15)
   - 사회초년생: 소득이 있는 업무에 종사한 기간이 총 5년 이내이며, 아래의 하나에 해당하는 사람
     1) 소득이 있는 업무에 종사하는 사람
     2) 퇴직한 후 1년이 지나지 않은 사람으로서 구직급여 수급자격을 인정받은 사람
     3) 예술인

3. **신혼부부·한부모가족 계층**
   - 신혼부부: 공고일 현재 혼인 중인 사람으로서 혼인기간이 7년 이내인 사람 또는 6세 이하 자녀(2018.05.16. 이후 출생)를 둔 사람
   - 예비신혼부부: 입주 전까지 혼인사실을 증명할 수 있는 사람
   - 한부모가족: 6세 이하 자녀(2018.05.16. 이후 출생)를 둔 한부모인 사람

4. **고령자**
   - 65세 이상의 사람 (1960.05.15. 이전 출생)

5. **주거급여수급자**
   - 「주거급여법」 제2조 제2호 및 제3호에 따른 수급권자 또는 수급자

신청자는 위의 자격 요건을 충족해야 하며, 세부적인 무주택·소득·자산 자격요건은 공고문 본문에서 확인해야 합니다.

참조 공고: 서울오류 행복주택 예비입주자 모집공고


In [9]:
# # 테스트 질문 4: 자산 기준
# question = "화성시 공고에 자산 기준이 나와있나요?"
# print(f"Q: {question}\n")
# print(f"A: {ask(question)}")

In [10]:
# # 테스트 질문 4: 자산 기준
# question = "화성시에 대한 공고가 있나요?"
# print(f"Q: {question}\n")
# print(f"A: {ask(question)}")

In [11]:
# 테스트 질문 4: 자산 기준
question = "화성시 공고의 임대보증금은 얼마야?"
print(f"Q: {question}\n")
print(f"A: {ask(question)}")

Q: 화성시 공고의 임대보증금은 얼마야?

[검색 결과: 26개 청크]
A: 화성시 행복주택의 임대보증금은 다음과 같습니다:

1. **화성비봉 A-2블록**:
   - 임대보증금: 14,620,000원 (대학생, 청년(소득無))
   - 임대보증금: 15,480,000원 (청년(소득有))

2. **화성상리 1블록**:
   - 임대보증금: 18,020,000원 (대학생, 청년(소득無))
   - 임대보증금: 19,080,000원 (청년(소득有))

3. **화성남양뉴타운 B-9블록**:
   - 임대보증금: 13,940,000원 (대학생, 청년(소득無))
   - 임대보증금: 14,760,000원 (청년(소득有))

4. **화성발안 A-1블록 (산업단지형 행복주택)**:
   - 임대보증금: 12,240,000원 (대학생, 청년(소득無))
   - 임대보증금: 12,960,000원 (청년(소득有))

각 블록에 따라 임대보증금이 다르므로, 신청자는 해당 블록의 조건을 확인해야 합니다.

참조 공고: [화성시 행복주택 입주자격완화 예비입주자 모집]


In [12]:
# question = "서울에 살고 있는 청년에게 적절한 공고 알려줘."
# print(f"Q: {question}\n")
# print(f"A: {ask(question)}")

In [13]:
# question = "지금 접수마감 안된 공고 있어?"
# print(f"Q: {question}\n")
# print(f"A: {ask(question)}")

## 5. 대화형 테스트

In [14]:
# # 대화형 테스트 (원하는 질문 입력)
# while True:
#     question = input("\n질문 (종료: q): ")
#     if question.lower() == 'q':
#         print("종료합니다.")
#         break
    
#     print(f"\nA: {ask(question)}")

## 6. 검색 결과 확인 (디버깅용)

In [15]:
# 검색 결과만 확인
query = "신혼부부 소득 기준"
query_embedding = embed_text(query)
results = hybrid_search(query, query_embedding, top_k=5)

print(f"검색 쿼리: {query}")
print(f"검색 결과: {len(results)}건\n")

for i, r in enumerate(results):
    chunk_id, chunk_text, elem_type, table_ctx, title, region, category, similarity = r
    print(f"[{i+1}] ID:{chunk_id} ({elem_type}) - 유사도: {similarity:.3f}")
    print(f"    공고: {title[:40]}...")
    if table_ctx:
        print(f"    컨텍스트: {table_ctx}")
    print(f"    내용: {chunk_text[:100].replace(chr(10), ' ')}...\n")

검색 쿼리: 신혼부부 소득 기준
검색 결과: 5건

[1] ID:1687 (table) - 유사도: 0.517
    공고: [신규모집]부천원종 A2블록 행복주택 입주자모집...
    컨텍스트: 적용 대상
    내용: * 7인이상의 가구는 6인가구 기준소득금액에 추가 1인당 평균금액 702,038원을 합산하여 산정 - 최종 소득 · 자산기준 (한도) - 자산기준 - 자동차가액 | 《 신혼부부·한...

[2] ID:1682 (text) - 유사도: 0.534
    공고: [신규모집]부천원종 A2블록 행복주택 입주자모집...
    내용: ## 신혼부부·한부모가족 계층 출생자녀 수에 따른 가산 소득·자산 기준  * 가산 적용 : 출산자녀 2인 이상 20% 가산, 출산자녀 1인 10% 가산...

[3] ID:1079 (text) - 유사도: 0.534
    공고: 서울오류 행복주택 예비입주자 모집공고...
    내용: ## 신혼부부·한부모가족 계층 출생자녀 수에 따른 가산 소득·자산 기준  * 가산 적용 : 출산자녀 2인 이상 20% 가산, 출산자녀 1인 10% 가산...

[4] ID:5 (text) - 유사도: 0.486
    공고: [정정공고]화성시 행복주택 입주자격완화 예비입주자 모집...
    내용: ## 소득요건 (전년도 도시근로자 가구원수별 가구당 월평균소득)  * 2순위 입주자의 갱신계약 시 완화된 소득요건을 기준으로 초과 비율에 따라 할증된 임대조건이 적용됩니다. * 3...

[5] ID:290 (text) - 유사도: 0.471
    공고: [정정공고]파주운정3 A24BL 영구임대주택 추가 입주자 모집공고...
    내용: ## 입주자격별 소득 및 자산 기준  * 출산(임신 중이거나 입양한 경우 포함)가구에 대한 소득 및 자산요건 완화 - 1) ‘23.3.28일 이후 출생한 자녀만 1명: 10%p 가...

